## Check List 1 - Robots
* **1.1. indy**  
  - joint_move_make_sure, move_joint_s_curve, move_joint_wp, grasp  
* **1.2. panda**  
  - joint_move_make_sure,  move_joint_s_curve,  move_joint_wp,  grasp  
* **1.3. combined_robot**  
  - joint_move_make_sure, move_joint_wp, grasp  

## set running directory to project source

In [1]:
import os
import numpy as np
import time
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## 1.1. indy
**Turn indy on, run controller program from rnb-control project**  
1. Turn ControlBox's power on.
2. Connect to ControlBox's ssh server 
```
ssh user@192.168.0.63
```
3. Stop the default program
```
sudo killall -9 UDEVMonitor && sudo killall -9 TaskMan
```
4. Start the rnb-control version control program.
```
cd ~/release/IndySDK_2.3.0.1/ && sudo ./TaskManager -j indyDeploy.json
```  
5. Connect Conty and reset the robot in Conty (There is jitter a issue without conty connection)

***WARNING: Clear the space around the robot! The robot will move.***

In [23]:
from pkg.controller.trajectory_client.indy_trajectory_client import IndyTrajectoryClient
INDY_IP = "192.168.0.63"
INDY_HOME = [0, 0, -np.pi / 2, 0, -np.pi / 2, 0]

##### create IndyTrajectoryClient

In [24]:
indy = IndyTrajectoryClient(server_ip=INDY_IP)

Connect: Server IP (192.168.0.63)


##### joint_move_make_sure()
  * This command will move indy to given position, using functions of IndyDCPClient.

In [25]:
indy.joint_move_make_sure(INDY_HOME)

Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)


##### move_joint_s_curve(qtar, N_div=100)
  * This command will move indy to given position, with a double S velocity profile. 
  * N_div is the number of steps to divide motion

In [26]:
indy.move_joint_s_curve(np.add(INDY_HOME, [0.2]*6), N_div=100)

Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)


##### move_joint_wp()
  * This command will move indy along given waypoint positions.
  * vel_lims & acc_lims are velocity and acceleration limits for the joints. They can be both scalar and vector, in radians.
  * vel_lims & acc_lims are approaximately applied. Check the source code for detail

In [73]:
indy.move_joint_wp(
    [np.add(INDY_HOME, [0]*6), 
     np.add(INDY_HOME, [0.4]*6), 
     np.add(INDY_HOME, [-0.4]*6), 
     np.array([0]*6),
     INDY_HOME
    ], 
    vel_lims=[2]*6, acc_lims=[4.0]*6)

Connect: Server IP (192.168.0.63)


##### grasp()
  * This command will move indy gripper, if one is attached.

In [7]:
indy.grasp(True)
time.sleep(0.5)
indy.grasp(False)
time.sleep(0.5)

Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)


## 1.2. panda
**Turn panda and host computer on, run controller program from rnb-control project**  
1. Turn panda robot and host computer power on
2. When panda is prepared, open Frank Emika Desk on a web browser (https://192.168.0.13/desk/)
3. On right side, click "unlock" icon to unlock joints.
4. Connect to host computer's ssh server 
```
ssh panda@192.168.0.172
```
5. Hold the enabling switch handle.
6. Run panda control program in ssh
```
roslaunch panda_control joint_control_rnb.launch robot_ip:=192.168.0.13 load_gripper:=false
```
7. When the robot stops due to collision or acceleration limit, repeat 5~6 to re-start the control program  

***WARNING: Clear the space around the robot! The robot will move.***

In [2]:
from pkg.controller.trajectory_client.panda_trajectory_client import PandaTrajectoryClient
PANDA_HOST_IP = "192.168.0.172" ## The host computer is the computer that runs control program
PANDA_ROBOT_IP = "192.168.0.13" ## The robot has it's own IP
PANDA_HOME = [0, -np.pi / 8, 0, -np.pi / 2, 0, np.pi / 2, np.pi / 2]

In [3]:
import rospy
rospy.init_node("panda_test") # ros node is initialized because panda gripper uses ros api. 

##### create PandaTrajectoryClient

In [4]:
panda = PandaTrajectoryClient(server_ip=PANDA_HOST_IP, robot_ip=PANDA_ROBOT_IP)

##### joint_move_make_sure()
  * This command will move panda to given position. panda uses move_joint_s_curve

In [9]:
panda.joint_move_make_sure(PANDA_HOME)

In [8]:
panda.move_joint_wp(
    [np.add(PANDA_HOME, [0]*7), 
     np.add(PANDA_HOME, [0.4]*7), 
     np.add(PANDA_HOME, [0]*7)], 
    vel_lims=np.array([1]*7), acc_lims=np.array([1.5]*7))

##### move_joint_s_curve(qtar, N_div=100)
  * This command will move panda to given position, with a double S velocity profile. 
  * N_div is the number of steps to divide motion

In [535]:
panda.move_joint_s_curve(np.add(PANDA_HOME, [0.2]*7), N_div=100)
panda.joint_move_make_sure(PANDA_HOME)

##### move_joint_wp()
  * This command will move panda along given waypoint positions.
  * vel_lims & acc_lims are velocity and acceleration limits for the joints. They can be both scalar and vector, in radians.
  * vel_lims & acc_lims are approaximately applied. Check the source code for detail

In [11]:
panda.move_joint_wp(
    [np.add(PANDA_HOME, [0]*7), 
     np.add(PANDA_HOME, [0.2]*7), 
     np.add(PANDA_HOME, [0.1]*7), 
     np.add(PANDA_HOME, [0.3]*7), 
     np.add(PANDA_HOME, [0.25]*7), 
     np.add(PANDA_HOME, [0.3]*7), 
     np.add(PANDA_HOME, [0.15]*7), 
     np.add(PANDA_HOME, [0.4]*7), 
     np.add(PANDA_HOME, [0.25]*7), 
     np.add(PANDA_HOME, [0.3]*7), 
     np.add(PANDA_HOME, [0.15]*7), 
     np.add(PANDA_HOME, [0.4]*7), 
     np.add(PANDA_HOME, [0.3]*7), 
     np.add(PANDA_HOME, [0.4]*7), 
     np.add(PANDA_HOME, [0.3]*7), 
     np.add(PANDA_HOME, [0.4]*7), 
     np.add(PANDA_HOME, [0.3]*7), 
     np.add(PANDA_HOME, [0.4]*7), 
     np.add(PANDA_HOME, [0.3]*7), 
     np.add(PANDA_HOME, [0.4]*7), 
     np.add(PANDA_HOME, [-0.3]*7), 
     np.add(PANDA_HOME, [-0.25]*7), 
     np.add(PANDA_HOME, [-0.3]*7), 
     np.add(PANDA_HOME, [-0.1]*7), 
     np.add(PANDA_HOME, [0]*7)], 
    vel_lims=np.array([1.5]*7), acc_lims=np.array([3.14]*7))

##### grasp()
  * This command will move panda gripper.

In [14]:
panda.grasp(True)
time.sleep(0.5)
panda.grasp(False)
time.sleep(0.5)

## 1.3. CombinedRobot
**Turn indy and panda on**  
***WARNING: Clear the space around the robots! The robots will move.***

In [3]:
from pkg.controller.combined_robot import CombinedRobot
from pkg.controller.robot_config import RobotConfig, RobotType
INDY_IP = "192.168.0.63"
PANDA_HOST_IP = "192.168.0.172" ## The host computer is the computer that runs control program
PANDA_ROBOT_IP = "192.168.0.13" ## The robot has it's own IP

In [4]:
import rospy
rospy.init_node("panda_test") # ros node is initialized because panda gripper uses ros api. 

##### create CombinedRobot instance

In [5]:
combined_robot = CombinedRobot(
    robots_on_scene=[
        RobotConfig(0, RobotType.indy7, ((0,-0.3,0), (0,0,0)),
                    INDY_IP),
        RobotConfig(1, RobotType.panda, ((0,0.3,0), (0,0,0)),
                    "{}/{}".format(PANDA_HOST_IP, PANDA_ROBOT_IP))], 
    connection_list=[False, False])

# CombinedRobot has combined home pose.
# Default home pose for each robot is defined in pkg.controller.robot_config.RobotSpecs
COMBINED_HOME = combined_robot.home_pose 
COMBINED_JOINT_NUM = combined_robot.joint_num

connection command:
indy0: [False, False]
Connect: Server IP (192.168.0.63)


##### reset_connection(bool, bool, ...)
* reset connection status as given as arguments

In [6]:
combined_robot.reset_connection(True, True)

connection command:
indy0: True
panda1: True


##### joint_move_make_sure()
  * This command will move both robots to given position, one-by-one.

In [18]:
combined_robot.joint_move_make_sure(COMBINED_HOME)

Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)


##### move_joint_wp()
  * This command will move the robots along given waypoint positions, one-by-one.
  * vel_scale & acc_scale are ratio to pre-defined velocity and acceleration limits in scaler. The pre-defined values are in pkg.controller.robot_config.RobotSpecs

In [19]:
combined_robot.move_joint_wp(
    [np.add(COMBINED_HOME, [0]*COMBINED_JOINT_NUM), 
     np.add(COMBINED_HOME, [0.2]*COMBINED_JOINT_NUM), 
     np.add(COMBINED_HOME, [-0.2]*COMBINED_JOINT_NUM), 
     np.add(COMBINED_HOME, [0]*COMBINED_JOINT_NUM)], 
    vel_scale=0.5, acc_scale=0.5)

Connect: Server IP (192.168.0.63)


##### grasp(robot1=bool, robot2=bool, ...)
  * This command will move each robot gripper.

In [21]:
combined_robot.grasp(indy0=True, panda1=True)
time.sleep(0.5)
combined_robot.grasp(indy0=False, panda1=False)
time.sleep(0.5)

Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)
